In [1]:
import os
import sys
import torch
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../voice_cloning')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../voice_cloning/Grad-TTS')))
from speaker_encoder import ecapa_tdnn
from model import tts, unit_encoder
from text.symbols import symbols
import params
from tqdm import tqdm

2025-04-08 16:38:30 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
/repo/voice-cloning/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ecapa_state_dict = torch.load('../voice_cloning/Grad-TTS/checkpts/speaker_encoder.pt')
grad_state_dict = torch.load('../voice_cloning/Grad-TTS/checkpts/grad_2_unit.pt')
ecapa = ecapa_tdnn.ECAPA_TDNN_SMALL_WTTH_PROJ()
add_blank = params.add_blank
nsymbols = len(symbols) + 1 if add_blank else len(symbols)
n_spks = params.n_spks
spk_emb_dim = params.spk_emb_dim
n_enc_channels = params.n_enc_channels
filter_channels = params.filter_channels
filter_channels_dp = params.filter_channels_dp
n_enc_layers = params.n_enc_layers
enc_kernel = params.enc_kernel
enc_dropout = params.enc_dropout
n_heads = params.n_heads
window_size = params.window_size

n_feats = params.n_feats
n_fft = params.n_fft
sample_rate = params.sample_rate
hop_length = params.hop_length
win_length = params.win_length
f_min = params.f_min
f_max = params.f_max

dec_dim = params.dec_dim
beta_min = params.beta_min
beta_max = params.beta_max
pe_scale = params.pe_scale

model = tts.GradTTS(nsymbols, n_spks, spk_emb_dim, n_enc_channels,
                    filter_channels, filter_channels_dp,
                    n_heads, n_enc_layers, enc_kernel, enc_dropout, window_size,
                    n_feats, dec_dim, beta_min, beta_max, pe_scale).cuda()
    
model.load_state_dict(torch.load('../voice_cloning/Grad-TTS/checkpts/trained_decoder.pt', map_location=lambda loc, storage: loc))

model.encoder = unit_encoder.UnitEncoder(nsymbols, n_feats, n_enc_channels, filter_channels,
                    filter_channels_dp, n_heads, n_enc_layers, enc_kernel,
                    enc_dropout, window_size, n_contentvec=768).cuda()
ecapa.load_state_dict(ecapa_state_dict)
model.load_state_dict(grad_state_dict)



/repo/voice-cloning/venv/lib/python3.12/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (256) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(


<All keys matched successfully>

In [3]:
model.spk_emb = ecapa
model.spk_emb.requires_grad = False
model.encoder.requires_grad = False
model.to(device)

GradTTS(
  (spk_emb): ECAPA_TDNN_SMALL_WTTH_PROJ(
    (spk): ECAPA_TDNN(
      (feature_extract): MelSpectrogram(
        (spectrogram): Spectrogram()
        (mel_scale): MelScale()
      )
      (instance_norm): InstanceNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (layer1): Conv1dReluBn(
        (conv): Conv1d(256, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (layer2): SE_Res2Block(
        (Conv1dReluBn1): Conv1dReluBn(
          (conv): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
          (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (Res2Conv1dReluBn): Res2Conv1dReluBn(
          (convs): ModuleList(
            (0-6): 7 x Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
          )
          (bns): ModuleList(
            (0-6): 7 x BatchNor

In [4]:
import torchaudio
import librosa
import IPython.display as ipd
source_path = "Amir2.wav"
wav, sr = librosa.load(source_path)
print('Source speech')
ipd.display(ipd.Audio(wav, rate=sr))
wav = torch.FloatTensor(wav).unsqueeze(0)

Source speech


In [5]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../voice_cloning/Grad-TTS/hifi-gan')))

from data import UnitMelSpeakerDataset, UnitMelSpeakerBatchCollate
from torch.utils.data import DataLoader
train_dataset = UnitMelSpeakerDataset('finetune.txt', '../voice_cloning/Grad-TTS/resources/cmu_dictionary', params.add_blank,
                                          params.n_fft, params.n_feats, params.sample_rate, params.hop_length,
                                          params.win_length, params.f_min, params.f_max)
batch_collate = UnitMelSpeakerBatchCollate()
loader = DataLoader(dataset=train_dataset, batch_size=1,
                        collate_fn=batch_collate, drop_last=True,
                        num_workers=1, shuffle=True)

In [6]:
resample_fn = torchaudio.transforms.Resample(sr, params.sample_rate)
spk = resample_fn(wav)
# torchaudio.save("Amir2.wav", spk, 24000)
spk.shape

torch.Size([1, 490177])

In [7]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=params.learning_rate)
iterations = 2000

In [8]:

model.train()
for iter in tqdm(range(iterations)):
    for batch in loader:
        model.zero_grad()
        x, x_lengths = batch['x'].cuda(), batch['x_lengths'].cuda()
        y, y_lengths = batch['y'].cuda(), batch['y_lengths'].cuda()
        dur_loss, prior_loss, diff_loss = model.compute_loss(x, x_lengths,
                                                                                        y, y_lengths,
                                                                    spk=spk.cuda(), out_size=params.out_size)
        loss = sum([dur_loss, prior_loss, diff_loss])
        loss.backward()
        optimizer.step()

  0%|          | 0/2000 [00:00<?, ?it/s]/repo/voice-cloning/venv/lib/python3.12/site-packages/torch/functional.py:709: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /pytorch/aten/src/ATen/native/SpectralOps.cpp:873.)
  return _VF.stft(  # type: ignore[attr-defined]
  0%|          | 1/2000 [00:00<32:12,  1.03it/s]/repo/voice-cloning/venv/lib/python3.12/site-packages/torch/functional.py:709: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /pytorch/aten/src/ATen/native/Spectra

In [9]:
import numpy as np
test_dataset = UnitMelSpeakerDataset('test.txt', '../voice_cloning/Grad-TTS/resources/cmu_dictionary', params.add_blank,
                                          params.n_fft, params.n_feats, params.sample_rate, params.hop_length,
                                          params.win_length, params.f_min, params.f_max)
x = test_dataset[0]['x'].unsqueeze(0)
x_lengths = torch.LongTensor([np.floor((x.shape[-1] - 400) / 320) + 1]).cuda()
x.shape, x_lengths

/repo/voice-cloning/venv/lib/python3.12/site-packages/torch/functional.py:709: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /pytorch/aten/src/ATen/native/SpectralOps.cpp:873.)
  return _VF.stft(  # type: ignore[attr-defined]


(torch.Size([1, 84800]), tensor([264], device='cuda:0'))

In [10]:
source_path = "/repo/voice-cloning/text_encoder_dataset/LibriTTS/train-clean-100/19/198/19_198_000000_000002.wav"
wav, sr = librosa.load(source_path)
print('Source speech')
ipd.display(ipd.Audio(wav, rate=sr))
wav = torch.FloatTensor(wav).unsqueeze(0)
spk2 = resample_fn(wav)
spk2.shape

Source speech


torch.Size([1, 127200])

In [11]:
y_enc, y_dec, attn = model(x.cuda(), x_lengths.cuda(), torch.tensor(50).cuda(), spk=spk.cuda())

In [12]:
from models import Generator as HiFiGAN
from env import AttrDict
import json
with open('/repo/voice-cloning/voice_cloning/Grad-TTS/checkpts/hifigan-config.json') as f:
    h = AttrDict(json.load(f))
hifigan = HiFiGAN(h)
hifigan.load_state_dict(torch.load('/repo/voice-cloning/voice_cloning/Grad-TTS/checkpts/hifigan.pt', map_location=lambda loc, storage: loc)['generator'])
_ = hifigan.cuda().eval()
hifigan.remove_weight_norm()

Removing weight norm...


/repo/voice-cloning/venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [13]:

with torch.no_grad():
    audio = hifigan.forward(y_dec).cpu().squeeze().clamp(-1, 1)
ipd.display(ipd.Audio(audio, rate=22050))

In [14]:
from model.text_encoder import TextEncoder
model.encoder = TextEncoder(model.n_vocab, model.n_feats, model.n_enc_channels, 
                                   model.filter_channels, model.filter_channels_dp, model.n_heads, 
                                   model.n_enc_layers, model.enc_kernel, model.enc_dropout, model.window_size)
model.encoder.load_state_dict(torch.load('/repo/voice-cloning/voice_cloning/Grad-TTS/checkpts/text_encoder.pt'))
model.encoder.to(device)

TextEncoder(
  (emb): Embedding(149, 192)
  (prenet): ConvReluNorm(
    (conv_layers): ModuleList(
      (0-2): 3 x Conv1d(192, 192, kernel_size=(5,), stride=(1,), padding=(2,))
    )
    (norm_layers): ModuleList(
      (0-2): 3 x LayerNorm()
    )
    (relu_drop): Sequential(
      (0): ReLU()
      (1): Dropout(p=0.5, inplace=False)
    )
    (proj): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
  )
  (encoder): Encoder(
    (drop): Dropout(p=0.1, inplace=False)
    (attn_layers): ModuleList(
      (0-5): 6 x MultiHeadAttention(
        (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
        (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
        (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
        (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
        (drop): Dropout(p=0.1, inplace=False)
      )
    )
    (norm_layers_1): ModuleList(
      (0-5): 6 x LayerNorm()
    )
    (ffn_layers): ModuleList(
      (0-5): 6 x FFN(
        (conv_1)

In [18]:
from data import TextMelSpeakerDataset
test_dataset_text = TextMelSpeakerDataset('test.txt', '../voice_cloning/Grad-TTS/resources/cmu_dictionary', params.add_blank,
                                          params.n_fft, params.n_feats, params.sample_rate, params.hop_length,
                                          params.win_length, params.f_min, params.f_max)
x = test_dataset_text[0]['x'].unsqueeze(0)
x_lengths = torch.LongTensor([x.shape[-1]]).cuda()
x.shape, x_lengths

(torch.Size([1, 97]), tensor([97], device='cuda:0'))

In [19]:
y_enc, y_dec, attn = model(x.cuda(), x_lengths.cuda(), torch.tensor(50).cuda(), spk=spk.cuda())

In [20]:
with torch.no_grad():
    audio = hifigan.forward(y_dec).cpu().squeeze().clamp(-1, 1)
ipd.display(ipd.Audio(audio, rate=22050))

In [22]:
torchaudio.save(
        "generated_audio.wav", 
        audio.unsqueeze(0),  # Ensure shape [1, samples] for single-channel
        sample_rate=22050    # Match HiFiGAN's output sample rate
    )